In [1]:
# === donnees_clients.py ===
import numpy as np

clients_raw = [
    ("Belhadi", "Soso", 32, "France", "epargne"),
    ("Soumare", "Hamidou", 45, "Belgique", "courant"),
    ("Mooken", "Jayson", 27, "France", "connecte"),
    ("Dupont", "Jean", 25, "France", "epargne"),
]

clients = {}

for i, client in enumerate(clients_raw, start=1):
    nom, prenom, age, pays, type_compte = client
    clients[i] = {
        "nom": nom,
        "prenom": prenom,
        "age": age,
        "pays": pays,
        "type_compte": type_compte
    }

soldes = np.random.randint(50, 5000, size=len(clients))
for i, solde in enumerate(soldes, start=1):
    clients[i]["solde"] = solde

liste_soldes = np.array([client["solde"] for client in clients.values()])
solde_moyen = np.mean(liste_soldes)
total_depots = np.sum(liste_soldes)
nb_soldes_inf_100 = np.sum(liste_soldes < 100)

print("Solde moyen :", solde_moyen)
print("Total des dépôts :", total_depots)
print("Nombre de clients avec un solde < 100 € :", nb_soldes_inf_100)

print("\nClients avec un solde supérieur à la moyenne :")
for client in clients.values():
    if client["solde"] > solde_moyen:
        print(f"- {client['prenom']} {client['nom']} : {client['solde']} €")

def get_clients():
    return clients

# === banque.py ===
from abc import ABC, abstractmethod

class OperationInvalide(Exception):
    def __init__(self, message):
        super().__init__(message)

class Compte(ABC):
    def __init__(self, numero, solde, proprietaire):
        self.__numero = numero
        self._solde = solde
        self.proprietaire = proprietaire

    @abstractmethod
    def deposer(self, montant):
        pass

    @abstractmethod
    def retirer(self, montant):
        pass

    def __str__(self):
        return f"Compte #{self.__numero} - Propriétaire : {self.proprietaire} - Solde : {self._solde} €"

    def __eq__(self, other):
        return self._solde == other._solde

    def __len__(self):
        return len(str(self._solde))

class Connectable:
    def connecter(self):
        print(f"{self.__class__.__name__} connecté avec succès.")

class HistoriqueMixin:
    def __init__(self):
        self._historique = []

    def afficher_historique(self):
        print("\nHistorique :")
        for ligne in self._historique:
            print("-", ligne)

def logger_operation(fonction):
    def wrapper(self, *args, **kwargs):
        resultat = fonction(self, *args, **kwargs)
        operation = f"{fonction.__name__} appelé avec {args}"
        self._historique.append(operation)
        return resultat
    return wrapper

class CompteCourant(Compte, HistoriqueMixin):
    def __init__(self, numero, solde, proprietaire):
        Compte.__init__(self, numero, solde, proprietaire)
        HistoriqueMixin.__init__(self)

    @logger_operation
    def deposer(self, montant):
        if montant <= 0:
            raise OperationInvalide("Le montant déposé doit être positif.")
        self._solde += montant
       
    @logger_operation
    def retirer(self, montant):
        if montant <= 0:
            raise OperationInvalide("Le montant retiré doit être positif.")
        if self._solde - montant < -500:
            raise OperationInvalide("Le découvert autorisé est dépassé.")
        self._solde -= montant

class CompteEpargne(Compte, HistoriqueMixin):
    def __init__(self, numero, solde, proprietaire, taux_interet):
        Compte.__init__(self, numero, solde, proprietaire)
        HistoriqueMixin.__init__(self)
        self.__taux_interet = taux_interet 

    @logger_operation
    def deposer(self, montant):
        if montant <= 0:
            raise OperationInvalide("Le montant déposé doit être supérieur à 0.")
        self._solde += montant

    @logger_operation
    def retirer(self, montant):
        if montant <= 0:
            raise OperationInvalide("Le montant retiré doit être supérieur à 0.")
        if self._solde - montant < 0:
            raise OperationInvalide("Solde insuffisant pour effectuer ce retrait.")
        self._solde -= montant

    @logger_operation
    def appliquer_interet(self):
        interet = self._solde * self.__taux_interet
        self._solde += interet
        self._historique.append(f"Intérêt de {interet:.2f} € appliqué.")
        return interet

class CompteConnecte(Compte, Connectable, HistoriqueMixin):
    def __init__(self, numero, solde, proprietaire):
        Compte.__init__(self, numero, solde, proprietaire)
        HistoriqueMixin.__init__(self)
        self.__en_ligne = False

    def connecter(self):
        self.__en_ligne = True
        print(f"\nCompte connecté avec succès.")

    @logger_operation
    def deposer(self, montant):
        if not self.__en_ligne:
            raise OperationInvalide("Le compte n'est pas connecté.")
        if montant <= 0:
            raise OperationInvalide("Le montant déposé doit être positif.")
        self._solde += montant

    @logger_operation
    def retirer(self, montant):
        if not self.__en_ligne:
            raise OperationInvalide("Le compte n'est pas connecté.")
        if montant <= 0:
            raise OperationInvalide("Le montant retiré doit être positif.")
        if self._solde - montant < 0:
            raise OperationInvalide("Solde insuffisant.")
        self._solde -= montant

def comparer_comptes(c1, c2):
    if c1 == c2:
        print("Les deux comptes ont le même solde.")
    elif c1._solde > c2._solde:
        print(f"{c1.proprietaire} a un solde plus élevé.")
    else:
        print(f"{c2.proprietaire} a un solde plus élevé.")

# === main.py ===
clients = get_clients()

compte = CompteCourant(numero=101, solde=100, proprietaire="Soso Belhadi")

try:
    compte.deposer(200)
    compte.retirer(700)
    compte.retirer(200)
except OperationInvalide as e:
    print("Erreur :", e)

print(compte)
compte.afficher_historique()

epargne = CompteEpargne(numero=202, solde=1000, proprietaire="Hamidou Soumare", taux_interet=0.03)

try:
    epargne.deposer(500)
    epargne.retirer(200)
    epargne.retirer(2000)
    epargne.appliquer_interet()
except OperationInvalide as e:
    print("Erreur :", e)

print(epargne)
epargne.afficher_historique()

connecte = CompteConnecte(numero=303, solde=500, proprietaire="Jayson Mooken")

try:
    connecte.deposer(100)
except OperationInvalide as e:
    print("Erreur :", e)

connecte.connecter()

try:
    connecte.deposer(150)
    connecte.retirer(200)
except OperationInvalide as e:
    print("Erreur :", e)

print(connecte)
connecte.afficher_historique()

print("\n--- Comparaison entre comptes ---")
comparer_comptes(compte, epargne)
comparer_comptes(epargne, connecte)


Solde moyen : 2052.75
Total des dépôts : 8211
Nombre de clients avec un solde < 100 € : 0

Clients avec un solde supérieur à la moyenne :
- Jayson Mooken : 3660 €
- Jean Dupont : 2669 €
Erreur : Le découvert autorisé est dépassé.
Compte #101 - Propriétaire : Soso Belhadi - Solde : -400 €

Historique :
- deposer appelé avec (200,)
- retirer appelé avec (700,)
Erreur : Solde insuffisant pour effectuer ce retrait.
Compte #202 - Propriétaire : Hamidou Soumare - Solde : 1300 €

Historique :
- deposer appelé avec (500,)
- retirer appelé avec (200,)
Erreur : Le compte n'est pas connecté.

Compte connecté avec succès.
Compte #303 - Propriétaire : Jayson Mooken - Solde : 450 €

Historique :
- deposer appelé avec (150,)
- retirer appelé avec (200,)

--- Comparaison entre comptes ---
Hamidou Soumare a un solde plus élevé.
Hamidou Soumare a un solde plus élevé.
